# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [3]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [5]:
# Your answer here
def get_row_count(wine_df):
    return len(wine_df)

row_count = get_row_count(wine_df)
print(f"The DataFrame has {row_count} rows.")

The DataFrame has 178 rows.


_(ii)_ How many variables (columns) does the dataset contain?

In [7]:
# Your answer here
def get_column_count(wine_df):
    return len(wine_df.columns)

column_count = get_column_count(wine_df)
print(f"The DataFrame has {column_count} columns.")

The DataFrame has 14 columns.


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [8]:
# Your answer here
def column_info(df, column_name):

  if column_name not in df.columns:
    raise ValueError(f"Column '{column_name}' not found in DataFrame.")

  column = df[column_name]
  data_type = column.dtype
  levels = column.unique().tolist()
  
  return {"data_type": data_type, "levels": levels}

column_name = 'class'
info = column_info(wine_df, column_name)
print(f"Information for column '{column_name}':")
print(f"Data type: {info['data_type']}")
print(f"Levels: {info['levels']}")

Information for column 'class':
Data type: int32
Levels: [0, 1, 2]



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [9]:
# Your answer here
def count_columns_except(df, column_to_exclude):

    if column_to_exclude not in df.columns:
        return 'Column does not exist'  
    return len(df.columns) - 1

predictor_var = count_columns_except(wine_df, "class")
print(f"Number of predictor variables: {predictor_var}")

Number of predictor variables: 13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [10]:
standardize_wine = wine_df.copy()

columns_to_exclude = ['class']

columns_to_scale = standardize_wine.columns.difference(columns_to_exclude)

scaler = StandardScaler()

standardize_wine[columns_to_scale] = scaler.fit_transform(wine_df[columns_to_scale])

standardize_wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009,0
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242,0
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148,0
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574,0
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.876275,2.974543,0.305159,0.301803,-0.332922,-0.985614,-1.424900,1.274310,-0.930179,1.142811,-1.392758,-1.231206,-0.021952,2
174,0.493343,1.412609,0.414820,1.052516,0.158572,-0.793334,-1.284344,0.549108,-0.316950,0.969783,-1.129518,-1.485445,0.009893,2
175,0.332758,1.744744,-0.389355,0.151661,1.422412,-1.129824,-1.344582,0.549108,-0.422075,2.224236,-1.612125,-1.485445,0.280575,2
176,0.209232,0.227694,0.012732,0.151661,1.422412,-1.033684,-1.354622,1.354888,-0.229346,1.834923,-1.568252,-1.400699,0.296498,2


In [ ]:
# # Select predictors (excluding the last column)
# predictors = wine_df.iloc[:, :-1]

# # Standardize the predictors
# scaler = StandardScaler()
# predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# # Display the head of the standardized predictors
# print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

> It is important to standardize the predictor variable because it enures fair treatment of all variables, prevent dominance by variables with larger scales, and facilitate more accurate and comparable model results.

(ii) Why did we elect not to standard our response variable `Class`?

> We did not standardize "Class" because it is our response variable

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

> Setting a seed is important because it allows reproduction of consitant results for testing or comparisons. The particular seed value is important because it will lead to a different random pattern that could have different results

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [11]:
# set a seed for reproducibility
np.random.seed(1)

# split the data into a training and testing set. hint: use train_test_split !

# Your code here ...
wine_train, wine_test = train_test_split(
    standardize_wine, train_size=0.75, stratify=standardize_wine["class"]
    )

In [12]:
#wine_train.info()

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [18]:
# Your code here 1...
knn = KNeighborsClassifier(n_neighbors=5)
knn

KNeighborsClassifier()

In [19]:
# Your code here 2...
parameter_grid = {
    "n_neighbors": range(1, 50, 1),
}

In [20]:
# Your code here 3...
wine_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)

In [ ]:
#make array for predicor variables
predictor_var = wine_df.columns[:13].to_numpy()
predictor_var

array(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'], dtype=object)

In [22]:
# Tried malic_acid before instad of flavanoids and it was .73
# Your code here 4...
wine_tune_grid.fit(
    wine_train[predictor_var],
    wine_train["class"]
)

accuracies_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracies_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005024,0.001207,0.005552,0.000917,1,{'n_neighbors': 1},1.0,1.000000,0.928571,0.923077,0.923077,0.923077,1.0,0.923077,0.923077,1.0,0.954396,0.037270,29
1,0.003775,0.000861,0.004714,0.001423,2,{'n_neighbors': 2},1.0,0.928571,0.857143,0.923077,0.923077,0.846154,1.0,0.923077,0.923077,1.0,0.932418,0.052128,48
2,0.004758,0.000930,0.005375,0.001808,3,{'n_neighbors': 3},1.0,0.928571,0.928571,0.846154,1.000000,0.846154,1.0,0.923077,0.923077,1.0,0.939560,0.057312,47
3,0.005560,0.001456,0.006271,0.003070,4,{'n_neighbors': 4},1.0,0.928571,0.857143,0.923077,0.923077,0.846154,1.0,0.846154,0.923077,1.0,0.924725,0.058255,49
4,0.006248,0.001496,0.006825,0.001599,5,{'n_neighbors': 5},1.0,0.928571,0.928571,1.000000,0.923077,0.846154,1.0,0.923077,0.923077,1.0,0.947253,0.048787,43
5,0.005711,0.001272,0.006170,0.001867,6,{'n_neighbors': 6},1.0,0.928571,0.928571,1.000000,0.923077,0.846154,1.0,1.000000,0.923077,1.0,0.954945,0.050406,25
6,0.005111,0.001541,0.005055,0.000737,7,{'n_neighbors': 7},1.0,0.928571,0.928571,1.000000,0.923077,0.846154,1.0,1.000000,0.923077,1.0,0.954945,0.050406,25
7,0.004467,0.001395,0.004873,0.000969,8,{'n_neighbors': 8},1.0,0.928571,0.928571,1.000000,0.923077,0.846154,1.0,1.000000,0.923077,1.0,0.954945,0.050406,25
8,0.004046,0.001090,0.004598,0.001114,9,{'n_neighbors': 9},1.0,1.000000,0.928571,1.000000,0.923077,0.846154,1.0,0.923077,0.923077,1.0,0.954396,0.050719,29
9,0.004273,0.000987,0.004072,0.000829,10,{'n_neighbors': 10},1.0,1.000000,0.928571,1.000000,0.923077,0.846154,1.0,0.923077,0.923077,1.0,0.954396,0.050719,29


In [23]:
wine_tune_grid.best_params_

{'n_neighbors': 27}

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [24]:
# Your code here... n=27

upd_knn = KNeighborsClassifier(n_neighbors=27)
upd_knn.fit(X=wine_train[predictor_var], y=wine_train['class'])

KNeighborsClassifier(n_neighbors=27)

In [25]:
wine_test["predicted"] = upd_knn.predict(wine_test[predictor_var])
wine_test[["class", "predicted"]]

,class,predicted
43,0,0
131,2,2
78,1,0
86,1,1
84,1,1
5,0,0
82,1,1
163,2,2
56,0,0
37,0,0


In [27]:
upd_knn.score(
    wine_test[predictor_var],
    wine_test["class"]
)

0.9333333333333333

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 05/18/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-6-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
